In [18]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# --- 1. 数据加载与预处理 ---

# a) 加载节点特征矩阵 X (来自FCA处理后的二元上下文)
features_path = '../data/iris.data.cleaned.csv'
x_numpy = np.loadtxt(features_path, delimiter=',')
x = torch.tensor(x_numpy, dtype=torch.float)

# b) 加载邻接矩阵 A+ (来自FCA概念格) 并转换为 edge_index
adj_matrix_path = '../data/iris_A_plus.csv'
a_plus_numpy = np.loadtxt(adj_matrix_path, delimiter=',')
a_plus = torch.tensor(a_plus_numpy, dtype=torch.float)
# ==========================================================
# ====================  【核心修改】 ====================
#              让邻接矩阵变得稀疏
# ==========================================================
# 打印原始的边数
print(f"原始邻接矩阵中的非零元素数: {torch.count_nonzero(a_plus)}")

# 方法1：设置一个阈值，比如只保留权重 > 0 的连接
threshold = 10
a_plus[a_plus <= threshold] = 0

# (可选) 移除自环 (一个节点到自身的连接)
a_plus.fill_diagonal_(0)

print(f"阈值化后邻接矩阵中的非零元素数: {torch.count_nonzero(a_plus)}")
# ==========================================================

# 转换成稀疏格式
edge_index, edge_attr = dense_to_sparse(a_plus)

# c) 从原始 iris.data 文件加载标签 y
labels_path = '../data/iris.data'
# 为 iris 数据集添加列名
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
iris_df = pd.read_csv(labels_path, header=None, names=column_names)

# 提取最后一列的物种名
species_labels = iris_df['species'].values

# 将字符串标签转换为数值 (e.g., 'Iris-setosa' -> 0)
encoder = LabelEncoder()
y_numpy = encoder.fit_transform(species_labels)
y = torch.tensor(y_numpy, dtype=torch.long)

# 确保节点特征数量和标签数量一致
# 注意：FCA去重后，节点数可能会减少，这里我们假设节点与原始数据行一一对应
# 如果节点数少于原始数据行数，需要根据去重逻辑对齐标签
num_nodes = x.shape[0]
if num_nodes != len(y):
    print(f"警告: 特征矩阵的节点数 ({num_nodes}) 与标签数 ({len(y)}) 不匹配。")
    # 假设FCA的去重保留了每个唯一行第一次出现的位置，我们据此对齐标签
    # 首先，获取去重前的原始二元上下文（需要Java代码生成或假设）
    # 为简化起见，此处我们假设节点与原始数据一一对应，并截取标签以匹配节点数
    # 在实际应用中，应保存去重时对象ID的映射关系
    y = y[:num_nodes]
    print(f"已将标签截取为 {len(y)} 个以匹配节点数。")


# d) 创建 PyTorch Geometric 的 Data 对象
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# e) 划分训练集和测试集 (4:1)
num_train = int(num_nodes * 0.8)
num_test = num_nodes - num_train

# 创建一个乱序的索引
indices = torch.randperm(num_nodes)

# 创建训练和测试掩码 (mask)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[indices[:num_train]] = True
test_mask[indices[num_train:]] = True

# 将掩码添加到 Data 对象中
data.train_mask = train_mask
data.test_mask = test_mask


print("--- 数据加载完成 ---")
print(data)
print(f"节点总数: {data.num_nodes}")
print(f"训练节点数: {data.train_mask.sum().item()}")
print(f"测试节点数: {data.test_mask.sum().item()}")
print(f"类别数: {len(np.unique(y_numpy))}")


# --- 2. GNN 模型定义 ---

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        # 定义第一个GCN层，隐藏层维度设为16
        self.conv1 = GCNConv(num_node_features, 16)
        # 定义第二个GCN层，输出维度为类别数
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        # 第一层 GCN + ReLU 激活函数
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Dropout 可选，用于防止过拟合
        x = F.dropout(x, p=0.5, training=self.training)
        # 第二层 GCN
        x = self.conv2(x, edge_index)
        # 输出层通常接 LogSoftmax 用于 NLLLoss，或直接输出 logits 用于 CrossEntropyLoss
        return x

# --- 3. 训练与评估 ---

# 实例化模型
model = GCN(num_node_features=data.num_node_features, num_classes=len(np.unique(y_numpy)))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    # 模型前向传播
    out = model(data.x, data.edge_index)
    # 只使用训练节点的输出来计算损失
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    # 在所有节点上进行预测
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    # 只使用测试节点的预测结果来计算准确率
    correct = pred[data.test_mask] == data.y[data.test_mask]
    acc = int(correct.sum()) / int(data.test_mask.sum())
    return acc

print("\n--- 开始训练 ---")
for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0:
        test_acc = test()
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')

final_test_acc = test()
print(f'--- 训练完成 ---')
print(f'最终测试集准确率: {final_test_acc:.4f}')

原始邻接矩阵中的非零元素数: 22500
阈值化后邻接矩阵中的非零元素数: 653
--- 数据加载完成 ---
Data(x=[150, 126], edge_index=[2, 653], edge_attr=[653], y=[150], train_mask=[150], test_mask=[150])
节点总数: 150
训练节点数: 120
测试节点数: 30
类别数: 3

--- 开始训练 ---
Epoch: 020, Loss: 0.1560, Test Accuracy: 0.9667
Epoch: 040, Loss: 0.0449, Test Accuracy: 0.9667
Epoch: 060, Loss: 0.0285, Test Accuracy: 0.9667
Epoch: 080, Loss: 0.0274, Test Accuracy: 0.9667
Epoch: 100, Loss: 0.0182, Test Accuracy: 0.9667
Epoch: 120, Loss: 0.0057, Test Accuracy: 0.9667
Epoch: 140, Loss: 0.0271, Test Accuracy: 0.9667
Epoch: 160, Loss: 0.0141, Test Accuracy: 0.9667
Epoch: 180, Loss: 0.0268, Test Accuracy: 0.9667
Epoch: 200, Loss: 0.0066, Test Accuracy: 0.9667
--- 训练完成 ---
最终测试集准确率: 0.9667
